In [47]:
import pandas as pd
import urllib.request, json 


url = 'https://gist.githubusercontent.com/EvanMisshula/019f1f9e4e52c632bf767bda18dd4f55/raw/36223c79d83e8e6606f9df3941f92c6c282133c8/nest.json'

with urllib.request.urlopen(url) as url:
    data = json.loads(url.read().decode())
    
df = pd.DataFrame(data)

In [44]:
# df['timeServedBinned'] = df['timeServedBinned'].str.replace('-', ' to ')

In [48]:
df.to_csv('C:/users/public/underCustody.csv', index=False)

### Load Geojson data 

In [1]:
import pandas as pd
import urllib.request, json 


url = 'https://gist.githubusercontent.com/aulichney/e39466ea9781fb262b190fb943003738/raw/7d6b8df0df7d5dfb3dfb1a46bdea5f133992a5e1/output.geojson'

with urllib.request.urlopen(url) as url:
    data = json.loads(url.read().decode())
    
df = pd.DataFrame(data["features"])[['id','type','properties','geometry']]
# convert column to the valid json format 
df['properties'] = df['properties'].apply(lambda x: json.dumps(x))
df['geometry'] = df['geometry'].apply(lambda x: json.dumps(x))

Then save the data to C:/users/public on windows. 

In [2]:
df.to_csv('C:/Users/Public/ny_state_map.csv', index=False)

Install psycopg2, python API for postgres.

In [16]:
!pip install psycopg2

Setup the connection to the database. I already created a database in my localhost called "cany".

In [88]:
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    database="cany",
    user="postgres",
    password="database1")

Store the geometry and properties data as JSON format in postgres. The JSON format must be valid for this to work.

In [89]:
cur = conn.cursor()
cur.execute("DROP TABLE IF EXISTS ny_state_geojson");
cur.execute("CREATE TABLE ny_state_geojson (id serial PRIMARY KEY, type varchar, properties json, geometry json)");
cur.execute("COPY ny_state_geojson (id, type, properties, geometry) \
                FROM 'C:/users/public/ny_state_map.csv' DELIMITER ',' \
            CSV HEADER;")
# Make the changes to the database persistent
conn.commit() 
# Close communication with the database
cur.close()
conn.close()